In [5]:
import teilchen as tn
import numpy as np
import matplotlib.pyplot as plt
import Konstanten as k


class Box:

    def __str__(self):
        return f"Box(xmin={self.xmin}, xmax={self.xmax}, "\
                   f"ymin={self.ymin}, ymax={self.ymax})"
    
    def __init__(self, xmin, xmax, ymin, ymax):
        self.xmin = xmin
        self.xmax = xmax
        self.ymin = ymin
        self.ymax = ymax
        self.Liste = []
        
    def add_tn(self, Teilchen):
        """
        Fügt ein Teilchen aus der Klasse tn zur Box hinzu
        """
        self.Liste.append(Teilchen)
        
    def bewegungs_dgl(self, s, Liste):
        """Differentialgleichung für N Teilchen, 
        die die Kraft auf ein Teilchen i beschreibt 
        (Gravitation und elektrische Abstoßung) 
    
        Argumente: 
        s: Statevektor des aktuellen Teilchens im Format [x, y, vx, vy]
        i: Index des aktuellen Teilchens
        self.Liste: Liste aller Teilchen in der Box.
    
        Returns: 
        Beschleunigungsvektor als numpy-Array, 
        der für den aktuellen Zeitschritt auf ein Teilchen wirkt.
        """ 
        
        #Einführung des Statevektors
        x, y, vx, vy = s
        
        #Koordinaten der Gravitationsbeschleunigung als konstanter Startwert
        ax = 0
        ay = k.g
    
        
        for andere in Liste:  
            if andere.x == x and andere.y == y: 
                continue 
                
            #Koordinaten des Abstands zu anderen Teilchen
            rx = x - andere.x   
            ry = y - andere.y 
            
            r_Betrag = np.sqrt(rx**2 + ry**2)
            r_hoch3 = r_Betrag**3
            
            #Berechnung der Abstoßungskraft (nur positiven Abstandsbeträgen)
            if r_hoch3 > 0:
                eKraft = k.q**2 / r_hoch3
                ax -= eKraft * rx
                ay -= eKraft * ry

        return np.array([vx, vy, ax, ay])

    
    def rk4_step(self, bewegungs_dgl, s, dt):
        
        """
        Berechnet approximativ aus der Bewegungsgleichung von oben
        einen aktualisierten Statevektoren für ein Teilchen
        Dabei wird z.B. auf den x-Wert des aktuellen Statevektoren
        der mit Runge-Kutta gemittelte vx-Wert multipliziert mit dt
        addiert. Eine simple Berechnung wie x_neu = x + vx * dt ist
        für unsere Simulation zu ungenau, da sich vx stetig ändert.
        Selbes gilt für vy bzw. die Werte von ax und ay.
    
        Argumente:
        bewegungs_dgl: oben beschrieben
        s: Statevektor des aktuellen Teilchens im Format [x,y,vx,vy]
        dt: Zeitschritt der Simulation
    
        Returns:
        aktualisierter Statevektor für ein Teilchen nach einem Zeitschritt
        """
        
        k1 = dt * bewegungs_dgl(s,Liste)
        k2 = dt * bewegungs_dgl(s + 0.5 * k1, Liste)  
        k3 = dt * bewegungs_dgl(s + 0.5 * k2, Liste)  
        k4 = dt * bewegungs_dgl(s + k3, Liste)  

    
        return s + k1 / 6 + k2 / 3 + k3 / 3 + k4 / 6  
    
    def reflexion(self):
        """
        Leitet eine Reflexion ein sobald die Koordinaten eines Teilchens
        mit denen der Boxgrenzen kollidieren.
        """
        for Teilchen in Liste:
            # Bedingung für rechte und linke Boxwand (vx wird umgekehrt)
            if tn.x <= self.xmin or tn.x >= self.xmax:
                tn.vx *= -1
            # Bedingung für obere und untere Boxwand (vy wird umgekehrt)
            if tn.y <= self.ymin or tn.y>= self.ymax:
                tn.vy *= -1
    
    def s_step(self, dt):
        """
        Aktualisiert die Werte des vorherigen Statevektors für alle Teilchen in der Box
        """
        for Teilchen in self.Liste:
            # aktuellen Statevektor für Teilchen
            s = np.array([Teilchen.x, Teilchen.y, Teilchen.vx, Teilchen.vy])
            # Berechnung des neuen Statevektors nach Runge-Kutta
            s_step = self.rk4_step(self.bewegungs_dgl, s, dt)
            # Aktualisiere Informationen für den Statevektor
            Teilchen.x, Teilchen.y, Teilchen.vx, Teilchen.vy = s_step
            
            # Ausführen der Reflexion
        self.reflexion()